In [49]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold

In [50]:
df = pd.read_csv("../../new/datasets/balanced_grouped.csv")

In [51]:
df

,0,1,2,3,4,5,6,7,8,9,...,335,336,337,338,339,340,341,342,group,id
0,0.015594,0.010444,0.021063,0.025866,0.023642,0.026157,0.032252,0.027204,0.012955,0.010038,...,0.822413,0.854666,0.898153,0.943331,0.963960,1.000000,0.989876,0.990877,9,1
1,0.331350,0.262977,0.212018,0.300518,0.377132,0.383542,0.231755,0.086373,0.090431,0.221658,...,0.622610,0.634173,0.721204,0.828365,0.825327,0.854831,0.801820,0.771081,6,1
2,0.134930,0.075620,0.062616,0.044667,0.121870,0.126496,0.103467,0.070279,0.098824,0.085445,...,0.934640,0.942650,0.890793,0.946226,0.917741,1.000000,0.951573,0.985645,5,1
3,0.109130,0.129643,0.087304,0.051216,0.052661,0.060745,0.052967,0.048967,0.062728,0.064739,...,0.846347,0.877817,0.918180,0.958194,0.972685,1.000000,0.981704,0.972349,7,1
4,0.224262,0.227079,0.146116,0.080234,0.091311,0.125116,0.114054,0.055488,0.044267,0.103987,...,0.032131,0.029224,0.037412,0.051124,0.052330,0.076105,0.075062,0.095761,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.522966,0.397591,0.313419,0.284905,0.271822,0.246843,0.243718,0.294817,0.378934,0.408562,...,0.707678,0.711110,0.724560,0.741992,0.744172,0.769042,0.766855,0.788741,3,3
2996,0.201920,0.160191,0.144714,0.130479,0.123315,0.146114,0.197060,0.225130,0.199775,0.155762,...,0.681672,0.674718,0.676407,0.681894,0.675200,0.689825,0.683288,0.699186,3,3
2997,0.000000,0.162607,0.413518,0.572730,0.515844,0.394520,0.409770,0.492245,0.425406,0.261534,...,0.840853,0.764674,0.685685,0.607052,0.532962,0.483535,0.458896,0.475790,5,3
2998,0.257628,0.097062,0.009845,0.092240,0.235695,0.254916,0.118284,0.000000,0.053772,0.193354,...,0.975907,0.949945,0.933002,0.920661,0.897001,0.901540,0.889620,0.910721,9,3


In [52]:
# Separamos las características y la variable objetivo
X = df.iloc[:, :-2]
y = df.iloc[:, -1] - 1
# y.iloc[:, 0] = y.iloc[:, 0] - 1
group = df.iloc[:, -2]

In [55]:
gkf = GroupKFold(n_splits=10)

# Guardar las métricas para calcular promedios
accuracies = []

# 4. Crear un DataFrame para almacenar las métricas
df_results = pd.DataFrame(columns=['Fold', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa'])

In [56]:
fold_number = 1
for train_index, test_index in gkf.split(X, y, group):
    # print(train_index , len(train_index))
    # print(test_index, len(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # crear el modelo
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # compilar el modelo
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # entrenar el modelo
    history = model.fit(X_train, y_train, epochs=100, batch_size=50, validation_data=(X_test, y_test))

    # predecir las probabilidades para los datos de prueba
    y_probs = model.predict(X_test)
    y_pred = np.argmax(y_probs, axis=-1)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

        # Calcular las métricas
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Guardar los resultados en el DataFrame
    df_results = df_results.append({
        'Fold': fold_number,
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Kappa': kappa
    }, ignore_index=True)
    fold_number += 1


Epoch 1/100
54/54 [==============================] - 1s 5ms/step - loss: 0.7397 - accuracy: 0.7162 - val_loss: 0.3938 - val_accuracy: 0.8994
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 0.5245 - accuracy: 0.7983 - val_loss: 0.2985 - val_accuracy: 0.9123
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 0.4436 - accuracy: 0.8202 - val_loss: 0.3170 - val_accuracy: 0.8831
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 0.3865 - accuracy: 0.8574 - val_loss: 0.2634 - val_accuracy: 0.9188
Epoch 5/100
54/54 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8878 - val_loss: 0.2034 - val_accuracy: 0.9221
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2971 - accuracy: 0.9012 - val_loss: 0.2012 - val_accuracy: 0.9286
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2889 - accuracy: 0.8978 - val_loss: 0.1655 - val_accuracy: 0.9383
Epoch 8/100
5

C:\Users\orest\AppData\Local\Temp\ipykernel_26516\202412937.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


54/54 [==============================] - 1s 5ms/step - loss: 0.7053 - accuracy: 0.7370 - val_loss: 0.8858 - val_accuracy: 0.5167
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 0.4696 - accuracy: 0.8333 - val_loss: 0.8388 - val_accuracy: 0.5400
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 0.4000 - accuracy: 0.8522 - val_loss: 0.7717 - val_accuracy: 0.5800
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.3725 - accuracy: 0.8641 - val_loss: 0.8460 - val_accuracy: 0.5933
Epoch 5/100
54/54 [==============================] - 0s 2ms/step - loss: 0.3223 - accuracy: 0.8874 - val_loss: 0.5933 - val_accuracy: 0.6400
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.2919 - accuracy: 0.8996 - val_loss: 0.4524 - val_accuracy: 0.7700
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.2574 - accuracy: 0.9215 - val_loss: 0.3474 - val_accuracy: 0.8933
Epoch 8/100
54/54 [======

C:\Users\orest\AppData\Local\Temp\ipykernel_26516\202412937.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


54/54 [==============================] - 1s 5ms/step - loss: 0.7213 - accuracy: 0.7152 - val_loss: 0.5651 - val_accuracy: 0.7500
Epoch 2/100
54/54 [==============================] - 0s 3ms/step - loss: 0.5136 - accuracy: 0.8052 - val_loss: 0.4834 - val_accuracy: 0.7833
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 0.4335 - accuracy: 0.8300 - val_loss: 0.3960 - val_accuracy: 0.8167
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 0.3854 - accuracy: 0.8619 - val_loss: 0.3335 - val_accuracy: 0.8700
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 0.3317 - accuracy: 0.8878 - val_loss: 0.2859 - val_accuracy: 0.8867
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 0.3000 - accuracy: 0.9004 - val_loss: 0.2501 - val_accuracy: 0.9267
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 0.2716 - accuracy: 0.9144 - val_loss: 0.2790 - val_accuracy: 0.8867
Epoch 8/100
54/54 [======

...

C:\Users\orest\AppData\Local\Temp\ipykernel_26516\202412937.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


55/55 [==============================] - 1s 6ms/step - loss: 0.7342 - accuracy: 0.7172 - val_loss: 0.3553 - val_accuracy: 0.8960
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5023 - accuracy: 0.8031 - val_loss: 0.4370 - val_accuracy: 0.8356
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4387 - accuracy: 0.8316 - val_loss: 0.3132 - val_accuracy: 0.9027
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4154 - accuracy: 0.8386 - val_loss: 0.3382 - val_accuracy: 0.8960
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8805 - val_loss: 0.3075 - val_accuracy: 0.8993
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.2994 - accuracy: 0.8953 - val_loss: 0.3319 - val_accuracy: 0.8960
Epoch 7/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2889 - accuracy: 0.9053 - val_loss: 0.2907 - val_accuracy: 0.9128
Epoch 8/100
55/55 [======

C:\Users\orest\AppData\Local\Temp\ipykernel_26516\202412937.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


55/55 [==============================] - 1s 6ms/step - loss: 0.6944 - accuracy: 0.7351 - val_loss: 0.3429 - val_accuracy: 0.9125
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5334 - accuracy: 0.7913 - val_loss: 0.2542 - val_accuracy: 0.9226
Epoch 3/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4564 - accuracy: 0.8306 - val_loss: 0.2517 - val_accuracy: 0.9158
Epoch 4/100
55/55 [==============================] - 0s 3ms/step - loss: 0.3832 - accuracy: 0.8672 - val_loss: 0.1706 - val_accuracy: 0.9259
Epoch 5/100
55/55 [==============================] - 0s 3ms/step - loss: 0.3553 - accuracy: 0.8775 - val_loss: 0.1657 - val_accuracy: 0.9529
Epoch 6/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2940 - accuracy: 0.9027 - val_loss: 0.1212 - val_accuracy: 0.9562
Epoch 7/100
55/55 [==============================] - 0s 3ms/step - loss: 0.2784 - accuracy: 0.9094 - val_loss: 0.1468 - val_accuracy: 0.9327
Epoch 8/100
55/55 [======

C:\Users\orest\AppData\Local\Temp\ipykernel_26516\202412937.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


In [57]:
df_results

,Fold,Accuracy,Precision,Recall,F1-Score,Kappa
0,1.0,0.944805,0.947488,0.944805,0.945072,0.917229
1,2.0,0.883333,0.891020,0.883333,0.882066,0.825000
2,3.0,0.923333,0.932514,0.923333,0.924476,0.885000
3,4.0,0.896667,0.899270,0.896667,0.894430,0.845000
4,5.0,0.926667,0.929290,0.926667,0.926388,0.890000
5,6.0,0.926421,0.931456,0.926421,0.926638,0.889623
6,7.0,0.919732,0.922921,0.919732,0.918512,0.879615
7,8.0,0.886288,0.897014,0.886288,0.882648,0.829484
8,9.0,0.922819,0.932910,0.922819,0.920468,0.884217
9,10.0,0.949495,0.951290,0.949495,0.949701,0.924242


In [58]:
# Calcular el promedio y la desviación estándar de cada métrica
metrics_mean = df_results[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa']].mean()
metrics_std = df_results[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Kappa']].std()


In [60]:
metrics_mean

Accuracy     0.917956
Precision    0.923517
Recall       0.917956
F1-Score     0.917040
Kappa        0.876941
dtype: float64

In [61]:
metrics_std

Accuracy     0.022543
Precision    0.020972
Recall       0.022543
F1-Score     0.023614
Kappa        0.033809
dtype: float64

# Save model

In [67]:
# Save the model to a single file
model.save( './rna_balance.keras', save_format='keras_v3')

# Suspected Symbiotic Stars v1

In [29]:
import numpy as np
from sklearn.metrics import confusion_matrix

df_sus_sy = pd.read_csv("../../new/candidate_symbiotic_stars_v1/normalized/Suspected_SY.csv", header=None)

# predecir las probabilidades para los datos de prueba
y_probs = model.predict(df_sus_sy)
y_pred = np.argmax(y_probs, axis=-1)
cm = confusion_matrix([0 for _ in range(len(df_sus_sy))], y_pred, labels=[0, 1, 2])
print(cm)

1/1 [==============================] - 0s 18ms/step
[[14  0  1]
 [ 0  0  0]
 [ 0  0  0]]


In [30]:
df_sus_sy_normalized = pd.read_csv("../../new/candidate_symbiotic_stars_v1/calibrated_data/Suspected_SY.csv")

df5 = pd.DataFrame(y_probs)
df5 = df5.round(4)
df5['label'] = y_pred
df5.insert(0, 'source_id', df_sus_sy_normalized['source_id'])
df5.head(5)

,source_id,0,1,2,label
0,4687286621186701568,0.9990,0.0008,0.0001,0
1,4651824725526390016,0.9098,0.0877,0.0025,0
2,3321366590173335424,0.9483,0.0410,0.0106,0
3,5410876219867043072,0.9999,0.0001,0.0000,0
4,3575939163051304192,0.9638,0.0333,0.0029,0


In [31]:
df6 = pd.read_csv('../../new/candidate_symbiotic_stars_v1/built_dataset/suspected_SY_dataset.csv')
df6.head(5)

,FIND_NAME,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,Gaia DR3
0,RAW 1691,LIN 521,C*,C,Gaia DR2 4687286621186701568,RAW 1691|LIN 521|2MASS J01183570-7242213|OGLE ...,C*|Em*|LP*|LP*|Em*|MIR|NIR|*|C*?|LP?,4687286621186701568
1,[BE74] 583,[BE74] 583,LongPeriodV*,G/Ke:,Gaia DR2 4651824725526390016,2MASS J05265014-7106350|EROS2-star lm058-2n-25...,LP*|Em*|NIR|V*|*,4651824725526390016
2,StHA 55,EM* StHA 55,Mira,NaN,Gaia DR3 3321366590173335424,IRAS 05440+0642|ASAS J054642+0643.7|ASAS J0546...,Mi*|LP*|V*|LP*|SB*|LP*|MIR|V*|Em*|NIR|*|C*?|IR...,3321366590173335424
3,ZZ CMi,V* ZZ CMi,LongPeriodV*,M6I-IIep,Gaia DR3 3155368612444708096,BD+09 1633|AN 306.1934|DO 2156|GCRV 4915|G...,LP*|NIR|V*|*|IR|LP?,3155368612444708096
4,WRAY 16−51,WRAY 16-51,LongPeriodV*,M4,Gaia DR2 5410876219860836224,IRAS 09316-4621|AKARI-IRC-V1 J0933295-463450|D...,LP*|NIR|MIR|Em*|PN|*|IR,5410876219867043072


In [32]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 7, 9, 10, 11, 12]]
df_filtered.head(5)

,FIND_NAME,MAIN_ID,Gaia DR3,0,1,2,label
0,RAW 1691,LIN 521,4687286621186701568,0.9990,0.0008,0.0001,0
1,[BE74] 583,[BE74] 583,4651824725526390016,0.9098,0.0877,0.0025,0
2,StHA 55,EM* StHA 55,3321366590173335424,0.9483,0.0410,0.0106,0
3,WRAY 16−51,WRAY 16-51,5410876219867043072,0.9999,0.0001,0.0000,0
4,NSV 05572,V* VX Crv,3575939163051304192,0.9638,0.0333,0.0029,0


In [33]:
out_name = 'rna_balanced.csv'
out_dir = '../../new/candidate_symbiotic_stars_v1/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)

# Suspected Symbiotic Stars v2

In [34]:
import numpy as np
from sklearn.metrics import confusion_matrix

df_sus_sy = pd.read_csv("../../new/candidate_symbiotic_stars_v2/normalized/Suspected_SY.csv", header=None)

# predecir las probabilidades para los datos de prueba
y_probs = model.predict(df_sus_sy)
y_pred = np.argmax(y_probs, axis=-1)
cm = confusion_matrix([0 for _ in range(len(df_sus_sy))], y_pred, labels=[0, 1, 2])
print(cm)

1/1 [==============================] - 0s 23ms/step
[[13  1  3]
 [ 0  0  0]
 [ 0  0  0]]


In [35]:
df_sus_sy_normalized = pd.read_csv("../../new/candidate_symbiotic_stars_v2/calibrated_data/Suspected_SY.csv")

df5 = pd.DataFrame(y_probs)
df5 = df5.round(4)
df5['label'] = y_pred
df5.insert(0, 'source_id', df_sus_sy_normalized['source_id'])
df5.head(5)

,source_id,0,1,2,label
0,6204217186929931520,0.9987,0.0013,0.0000,0
1,4061952680197028224,0.9085,0.0881,0.0034,0
2,670455944074475008,0.3494,0.0075,0.6431,2
3,4068755633500598272,0.0110,0.0548,0.9343,2
4,2060829659152816768,0.0179,0.8098,0.1724,1


In [36]:
df6 = pd.read_csv('../../new/candidate_symbiotic_stars_v2/built_dataset/suspected_SY_dataset.csv')
df6.head(5)

,FIND_NAME,MAIN_ID,OTYPE,SP_TYPE,ID_Gaia,IDS,OTYPES,Gaia DR3
0,V748 Cen,V* V748 Cen,EclBin,Ae,Gaia DR3 6204217186929931520,CD-32 10517|ALS 18924|CRTS J145936.6-332503|CS...,EB*|Ro*|NIR|V*|Em*|*,6204217186929931520
1,WRAY 16-294,WRAY 16-294,LongPeriodV*,K5,Gaia DR2 4061952680197028224,2MASS J17391381-2538050|DENIS J173913.8-253805...,LP*|PN|NIR|Em*|*|C*?|ISM|LP?,4061952680197028224
2,DASCH J075731.1+201735,ASAS J075731+2017.6,LongPeriodV*,M0III,Gaia DR2 670455944074475008,2MASS J07573112+2017347|ASAS J075731+2017.6|DA...,SB*|LP*|NIR|V*|*|Opt,670455944074475008
3,ASAS J174600-2321.3,ASAS J174600-2321.3,LongPeriodV*_Candidate,F0I,Gaia DR2 4068755633500598272,2MASS J17460018-2321163|ASAS J174600-2321.3|ER...,NIR|V*|*|LP?,4068755633500598272
4,IPHASJ201550.96+373004.2,IRAS 20140+3720,PlanetaryNeb_Candidate,NaN,Gaia DR2 2060829659152816768,2MASS J20155096+3730042|AKARI-IRC-V1 J2015509+...,NIR|*|C*?|IR|LP?|PN?,2060829659152816768


In [37]:
# Filtro de data frames
df_filtered = df6.merge(df5, left_on=['Gaia DR3'], right_on=['source_id'], how='inner', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'both']
df_filtered = df_filtered.iloc[:, [0, 1, 7, 9, 10, 11, 12]]
df_filtered.head(5)

,FIND_NAME,MAIN_ID,Gaia DR3,0,1,2,label
0,V748 Cen,V* V748 Cen,6204217186929931520,0.9987,0.0013,0.0000,0
1,WRAY 16-294,WRAY 16-294,4061952680197028224,0.9085,0.0881,0.0034,0
2,DASCH J075731.1+201735,ASAS J075731+2017.6,670455944074475008,0.3494,0.0075,0.6431,2
3,ASAS J174600-2321.3,ASAS J174600-2321.3,4068755633500598272,0.0110,0.0548,0.9343,2
4,IPHASJ201550.96+373004.2,IRAS 20140+3720,2060829659152816768,0.0179,0.8098,0.1724,1


In [38]:
out_name = 'rna_balanced.csv'
out_dir = '../../new/candidate_symbiotic_stars_v2/output'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

fullname = os.path.join(out_dir, out_name)
df_filtered.to_csv(fullname, header=True, index=False)